In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
import prediction
from utils import load_data,get_data_day
import logging
logging.basicConfig(level=logging.INFO)
%matplotlib inline
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking

Using TensorFlow backend.


In [4]:
data=pd.read_csv('fillna/O3.csv')
del data['Unnamed: 0']
data['date']=pd.to_datetime(data['date'])

In [11]:
groups=data.groupby(['idPolair'])

In [16]:
groups.get_group(7001)[['date','O3']].set_index(['date']).resample('d').mean()

,O3
date,
2012-01-01,41.375000
2012-01-02,59.458333
2012-01-03,61.625000
2012-01-04,62.041667
2012-01-05,68.083333
2012-01-06,41.916667
2012-01-07,41.291667
2012-01-08,57.583333
2012-01-09,44.625000


In [17]:
def seq(siko):
    x=pd.concat([siko['O3'].shift(9),siko['O3'].shift(8),siko['O3'].shift(7),siko['O3'].shift(6),siko['O3'].shift(5),siko['O3'].shift(4),siko['O3'].shift(3),siko['O3'].shift(2),siko['O3'].shift(1),siko['O3'],siko['O3'].shift(-1),siko['O3'].shift(-2)],axis=1
                ,keys=['t-9','t-8','t-7','t-6','t-5','t-4','t-3','t-2','t-1','t','t+1','t+2'])
    return x

In [24]:
models=dict({})

for name,g in groups:
    print(name)
    g=g[['date','O3']].set_index(['date']).resample('d').mean()
    g=seq(g).dropna()
    model = Sequential()
    model.add(LSTM(100,input_shape=(9,1)))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer='adam')
    Xtrain=np.array(g.iloc[:,:9])
    Xtrain=Xtrain.reshape(len(Xtrain), 9,1)
    ytrain=np.array(g.iloc[:,9:])
    ytrain=ytrain.reshape(len(ytrain),3)
    model.fit(Xtrain, ytrain, epochs=100, verbose=2,batch_size=32)
    models[name]=model

7001.0
Epoch 1/100
 - 2s - loss: 3591.1754
Epoch 2/100
 - 1s - loss: 2735.3421
Epoch 3/100
 - 1s - loss: 2121.7168
Epoch 4/100
 - 1s - loss: 1675.2710
Epoch 5/100
 - 1s - loss: 1351.7256
Epoch 6/100
 - 1s - loss: 1109.4153
Epoch 7/100
 - 1s - loss: 919.3290
Epoch 8/100
 - 1s - loss: 777.7197
Epoch 9/100
 - 1s - loss: 673.4869
Epoch 10/100
 - 1s - loss: 597.0015
Epoch 11/100
 - 1s - loss: 540.8761
Epoch 12/100
 - 1s - loss: 490.5261
Epoch 13/100
 - 1s - loss: 432.7435
Epoch 14/100
 - 1s - loss: 391.9414
Epoch 15/100
 - 1s - loss: 362.0626
Epoch 16/100
 - 1s - loss: 339.5963
Epoch 17/100
 - 1s - loss: 322.4453
Epoch 18/100
 - 1s - loss: 307.5418
Epoch 19/100
 - 1s - loss: 295.5562
Epoch 20/100
 - 1s - loss: 287.1001
Epoch 21/100
 - 1s - loss: 278.2071
Epoch 22/100
 - 1s - loss: 269.7975
Epoch 23/100
 - 1s - loss: 265.1577
Epoch 24/100
 - 1s - loss: 261.0634
Epoch 25/100
 - 1s - loss: 256.6838
Epoch 26/100
 - 1s - loss: 254.4130
Epoch 27/100
 - 1s - loss: 251.9684
Epoch 28/100
 - 1s - los

 - 2s - loss: 261.7243
Epoch 29/100
 - 1s - loss: 261.3169
Epoch 30/100
 - 1s - loss: 258.2921
Epoch 31/100
 - 2s - loss: 258.5291
Epoch 32/100
 - 1s - loss: 256.7590
Epoch 33/100
 - 1s - loss: 254.8480
Epoch 34/100
 - 1s - loss: 254.5058
Epoch 35/100
 - 1s - loss: 254.1900
Epoch 36/100
 - 1s - loss: 253.0062
Epoch 37/100
 - 1s - loss: 250.6759
Epoch 38/100
 - 1s - loss: 253.7147
Epoch 39/100
 - 1s - loss: 252.1905
Epoch 40/100
 - 1s - loss: 249.7383
Epoch 41/100
 - 1s - loss: 249.2620
Epoch 42/100
 - 1s - loss: 249.5831
Epoch 43/100
 - 1s - loss: 247.7210
Epoch 44/100
 - 1s - loss: 248.5275
Epoch 45/100
 - 1s - loss: 246.6511
Epoch 46/100
 - 1s - loss: 246.3357
Epoch 47/100
 - 1s - loss: 246.5091
Epoch 48/100
 - 1s - loss: 245.3357
Epoch 49/100
 - 1s - loss: 245.1269
Epoch 50/100
 - 1s - loss: 245.0617
Epoch 51/100
 - 1s - loss: 243.5997
Epoch 52/100
 - 1s - loss: 243.0245
Epoch 53/100
 - 1s - loss: 242.6503
Epoch 54/100
 - 1s - loss: 242.5944
Epoch 55/100
 - 1s - loss: 241.8463
Epoch

 - 1s - loss: 9.6032e-10
Epoch 61/100
 - 1s - loss: 9.1432e-10
Epoch 62/100
 - 1s - loss: 8.8599e-10
Epoch 63/100
 - 1s - loss: 8.6450e-10
Epoch 64/100
 - 1s - loss: 8.1277e-10
Epoch 65/100
 - 1s - loss: 7.3341e-10
Epoch 66/100
 - 1s - loss: 6.9623e-10
Epoch 67/100
 - 1s - loss: 6.9454e-10
Epoch 68/100
 - 1s - loss: 6.6910e-10
Epoch 69/100
 - 1s - loss: 5.9893e-10
Epoch 70/100
 - 1s - loss: 5.5284e-10
Epoch 71/100
 - 1s - loss: 5.4242e-10
Epoch 72/100
 - 1s - loss: 5.3516e-10
Epoch 73/100
 - 1s - loss: 4.9659e-10
Epoch 74/100
 - 1s - loss: 4.4789e-10
Epoch 75/100
 - 1s - loss: 4.1570e-10
Epoch 76/100
 - 1s - loss: 4.1026e-10
Epoch 77/100
 - 1s - loss: 3.9587e-10
Epoch 78/100
 - 1s - loss: 3.7436e-10
Epoch 79/100
 - 1s - loss: 3.6140e-10
Epoch 80/100
 - 1s - loss: 3.4806e-10
Epoch 81/100
 - 1s - loss: 3.1825e-10
Epoch 82/100
 - 1s - loss: 3.1161e-10
Epoch 83/100
 - 1s - loss: 3.0797e-10
Epoch 84/100
 - 1s - loss: 2.9830e-10
Epoch 85/100
 - 1s - loss: 2.9320e-10
Epoch 86/100
 - 1s - loss

Epoch 86/100
 - 1s - loss: 178.5501
Epoch 87/100
 - 1s - loss: 178.3050
Epoch 88/100
 - 1s - loss: 178.4217
Epoch 89/100
 - 1s - loss: 177.1514
Epoch 90/100
 - 1s - loss: 178.0871
Epoch 91/100
 - 1s - loss: 177.1536
Epoch 92/100
 - 1s - loss: 177.8263
Epoch 93/100
 - 1s - loss: 175.9405
Epoch 94/100
 - 1s - loss: 178.0937
Epoch 95/100
 - 1s - loss: 175.6909
Epoch 96/100
 - 1s - loss: 175.6808
Epoch 97/100
 - 1s - loss: 175.3998
Epoch 98/100
 - 1s - loss: 174.3811
Epoch 99/100
 - 1s - loss: 175.4731
Epoch 100/100
 - 1s - loss: 175.3148
7014.0
Epoch 1/100
 - 3s - loss: 3184.6757
Epoch 2/100
 - 1s - loss: 2532.1775
Epoch 3/100
 - 1s - loss: 1999.2290
Epoch 4/100
 - 1s - loss: 1545.5535
Epoch 5/100
 - 1s - loss: 1199.0828
Epoch 6/100
 - 1s - loss: 945.0024
Epoch 7/100
 - 1s - loss: 775.5395
Epoch 8/100
 - 1s - loss: 655.0178
Epoch 9/100
 - 1s - loss: 564.1453
Epoch 10/100
 - 1s - loss: 485.0646
Epoch 11/100
 - 1s - loss: 413.5214
Epoch 12/100
 - 1s - loss: 355.6895
Epoch 13/100
 - 1s - los

 - 1s - loss: 271.9652
Epoch 14/100
 - 1s - loss: 252.6808
Epoch 15/100
 - 1s - loss: 237.9072
Epoch 16/100
 - 1s - loss: 226.9304
Epoch 17/100
 - 1s - loss: 217.3890
Epoch 18/100
 - 1s - loss: 209.8874
Epoch 19/100
 - 1s - loss: 205.3273
Epoch 20/100
 - 1s - loss: 200.2684
Epoch 21/100
 - 1s - loss: 195.5240
Epoch 22/100
 - 1s - loss: 195.5869
Epoch 23/100
 - 1s - loss: 191.1777
Epoch 24/100
 - 1s - loss: 188.7449
Epoch 25/100
 - 1s - loss: 187.4250
Epoch 26/100
 - 1s - loss: 184.7843
Epoch 27/100
 - 1s - loss: 185.3570
Epoch 28/100
 - 1s - loss: 184.6968
Epoch 29/100
 - 1s - loss: 182.8701
Epoch 30/100
 - 1s - loss: 181.5361
Epoch 31/100
 - 1s - loss: 180.8085
Epoch 32/100
 - 1s - loss: 180.0721
Epoch 33/100
 - 1s - loss: 179.1178
Epoch 34/100
 - 1s - loss: 179.1301
Epoch 35/100
 - 1s - loss: 178.0700
Epoch 36/100
 - 1s - loss: 178.2369
Epoch 37/100
 - 1s - loss: 177.6641
Epoch 38/100
 - 2s - loss: 176.4688
Epoch 39/100
 - 1s - loss: 176.8523
Epoch 40/100
 - 1s - loss: 175.7598
Epoch

...

Epoch 60/100
 - 1s - loss: 199.7300
Epoch 61/100
 - 1s - loss: 199.1493
Epoch 62/100
 - 1s - loss: 198.6615
Epoch 63/100
 - 1s - loss: 198.6618
Epoch 64/100
 - 1s - loss: 198.8902
Epoch 65/100
 - 1s - loss: 197.2581
Epoch 66/100
 - 1s - loss: 196.4560
Epoch 67/100
 - 1s - loss: 197.7015
Epoch 68/100
 - 1s - loss: 199.1853
Epoch 69/100
 - 1s - loss: 196.2316
Epoch 70/100
 - 1s - loss: 197.1635
Epoch 71/100
 - 1s - loss: 196.8586
Epoch 72/100
 - 1s - loss: 195.0971
Epoch 73/100
 - 1s - loss: 194.4257
Epoch 74/100
 - 1s - loss: 196.6923
Epoch 75/100
 - 1s - loss: 195.2896
Epoch 76/100
 - 1s - loss: 193.7230
Epoch 77/100
 - 1s - loss: 192.6720
Epoch 78/100
 - 1s - loss: 193.0708
Epoch 79/100
 - 1s - loss: 193.2901
Epoch 80/100
 - 1s - loss: 193.0241
Epoch 81/100
 - 1s - loss: 193.2662
Epoch 82/100
 - 1s - loss: 191.9297
Epoch 83/100
 - 1s - loss: 192.8057
Epoch 84/100
 - 1s - loss: 193.6874
Epoch 85/100
 - 1s - loss: 191.3429
Epoch 86/100
 - 1s - loss: 193.4899
Epoch 87/100
 - 1s - loss: 1

Epoch 93/100
 - 1s - loss: 235.2856
Epoch 94/100
 - 1s - loss: 236.2107
Epoch 95/100
 - 1s - loss: 235.2325
Epoch 96/100
 - 1s - loss: 234.3397
Epoch 97/100
 - 1s - loss: 233.4684
Epoch 98/100
 - 1s - loss: 234.4639
Epoch 99/100
 - 1s - loss: 234.0541
Epoch 100/100
 - 1s - loss: 233.5958
33367.0
Epoch 1/100
 - 11s - loss: 667.9547
Epoch 2/100
 - 1s - loss: 451.4648
Epoch 3/100
 - 1s - loss: 321.6872
Epoch 4/100
 - 1s - loss: 227.7552
Epoch 5/100
 - 1s - loss: 158.2664
Epoch 6/100
 - 1s - loss: 107.4291
Epoch 7/100
 - 1s - loss: 71.0258
Epoch 8/100
 - 1s - loss: 45.6473
Epoch 9/100
 - 1s - loss: 28.4734
Epoch 10/100
 - 1s - loss: 17.2250
Epoch 11/100
 - 1s - loss: 10.1005
Epoch 12/100
 - 1s - loss: 5.7478
Epoch 13/100
 - 1s - loss: 3.1824
Epoch 14/100
 - 1s - loss: 1.7258
Epoch 15/100
 - 1s - loss: 0.9293
Epoch 16/100
 - 1s - loss: 0.5103
Epoch 17/100
 - 1s - loss: 0.2981
Epoch 18/100
 - 1s - loss: 0.1952
Epoch 19/100
 - 1s - loss: 0.1471
Epoch 20/100
 - 1s - loss: 0.1257
Epoch 21/100
 

Epoch 22/100
 - 1s - loss: 329.6145
Epoch 23/100
 - 1s - loss: 308.3846
Epoch 24/100
 - 1s - loss: 289.1905
Epoch 25/100
 - 1s - loss: 271.1284
Epoch 26/100
 - 1s - loss: 253.5695
Epoch 27/100
 - 1s - loss: 237.0467
Epoch 28/100
 - 1s - loss: 221.8036
Epoch 29/100
 - 1s - loss: 206.1300
Epoch 30/100
 - 1s - loss: 190.7828
Epoch 31/100
 - 1s - loss: 174.9836
Epoch 32/100
 - 1s - loss: 161.7465
Epoch 33/100
 - 1s - loss: 149.4556
Epoch 34/100
 - 1s - loss: 138.6105
Epoch 35/100
 - 1s - loss: 128.3861
Epoch 36/100
 - 1s - loss: 119.2729
Epoch 37/100
 - 1s - loss: 110.8725
Epoch 38/100
 - 1s - loss: 102.9691
Epoch 39/100
 - 1s - loss: 96.0292
Epoch 40/100
 - 1s - loss: 89.9714
Epoch 41/100
 - 1s - loss: 84.0158
Epoch 42/100
 - 1s - loss: 78.7835
Epoch 43/100
 - 1s - loss: 73.8036
Epoch 44/100
 - 1s - loss: 69.9994
Epoch 45/100
 - 1s - loss: 66.0040
Epoch 46/100
 - 1s - loss: 62.4838
Epoch 47/100
 - 1s - loss: 59.9885
Epoch 48/100
 - 1s - loss: 56.3539
Epoch 49/100
 - 1s - loss: 53.8207
Epo

Epoch 51/100
 - 1s - loss: 214.8338
Epoch 52/100
 - 1s - loss: 214.3878
Epoch 53/100
 - 1s - loss: 214.8658
Epoch 54/100
 - 1s - loss: 214.6036
Epoch 55/100
 - 1s - loss: 213.0755
Epoch 56/100
 - 1s - loss: 214.5155
Epoch 57/100
 - 1s - loss: 215.1241
Epoch 58/100
 - 1s - loss: 214.2990
Epoch 59/100
 - 1s - loss: 212.6309
Epoch 60/100
 - 1s - loss: 213.6480
Epoch 61/100
 - 1s - loss: 213.0909
Epoch 62/100
 - 1s - loss: 212.0040
Epoch 63/100
 - 1s - loss: 212.7306
Epoch 64/100
 - 1s - loss: 212.6218
Epoch 65/100
 - 1s - loss: 211.6546
Epoch 66/100
 - 1s - loss: 212.3482
Epoch 67/100
 - 1s - loss: 211.3138
Epoch 68/100
 - 1s - loss: 210.4864
Epoch 69/100
 - 1s - loss: 213.0774
Epoch 70/100
 - 1s - loss: 211.7212
Epoch 71/100
 - 1s - loss: 211.9584
Epoch 72/100
 - 1s - loss: 209.5892
Epoch 73/100
 - 1s - loss: 209.5766
Epoch 74/100
 - 1s - loss: 209.7886
Epoch 75/100
 - 1s - loss: 210.0145
Epoch 76/100
 - 1s - loss: 208.5543
Epoch 77/100
 - 1s - loss: 208.6909
Epoch 78/100
 - 1s - loss: 2

Epoch 85/100
 - 1s - loss: 81.5770
Epoch 86/100
 - 1s - loss: 82.1250
Epoch 87/100
 - 1s - loss: 81.7298
Epoch 88/100
 - 1s - loss: 82.3066
Epoch 89/100
 - 1s - loss: 81.4181
Epoch 90/100
 - 1s - loss: 82.0997
Epoch 91/100
 - 1s - loss: 82.5967
Epoch 92/100
 - 1s - loss: 83.0374
Epoch 93/100
 - 1s - loss: 81.4533
Epoch 94/100
 - 1s - loss: 80.9685
Epoch 95/100
 - 1s - loss: 80.6553
Epoch 96/100
 - 1s - loss: 80.9649
Epoch 97/100
 - 1s - loss: 81.7287
Epoch 98/100
 - 1s - loss: 81.7014
Epoch 99/100
 - 1s - loss: 81.0705
Epoch 100/100
 - 1s - loss: 83.9644
36019.0
Epoch 1/100
 - 10s - loss: 2863.0342
Epoch 2/100
 - 1s - loss: 2152.6923
Epoch 3/100
 - 1s - loss: 1692.8266
Epoch 4/100
 - 1s - loss: 1373.8241
Epoch 5/100
 - 1s - loss: 1143.2234
Epoch 6/100
 - 1s - loss: 967.2637
Epoch 7/100
 - 1s - loss: 828.7091
Epoch 8/100
 - 1s - loss: 715.3529
Epoch 9/100
 - 1s - loss: 626.5191
Epoch 10/100
 - 1s - loss: 553.1413
Epoch 11/100
 - 1s - loss: 493.3820
Epoch 12/100
 - 1s - loss: 443.9134
Ep

In [25]:
for name,g in groups:
    print(name)
    g=g[['date','O3']].set_index(['date']).resample('d').mean()
    g=seq(g).dropna()
    model = models[name]
    Xtrain=np.array(g.iloc[:,:9])
    Xtrain=Xtrain.reshape(len(Xtrain), 9,1)
    ytrain=np.array(g.iloc[:,9:])
    ytrain=ytrain.reshape(len(ytrain),3)
    model.fit(Xtrain, ytrain, epochs=1, verbose=2,batch_size=1)
    models[name]=model

7001.0


KeyError: 7001.0

In [26]:
models.keys()

dict_keys([])